In [1]:
!pip install tensorflow kagglehub

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import mixed_precision

In [3]:
# Set mixed precision policy
mixed_precision.set_global_policy('mixed_float16')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
random_rotation = tf.keras.layers.RandomRotation(0.2, dtype=tf.float32)
random_zoom = tf.keras.layers.RandomZoom(0.2, dtype=tf.float32)
random_flip = tf.keras.layers.RandomFlip("horizontal", dtype=tf.float32)
random_contrast = tf.keras.layers.RandomContrast(0.3, dtype=tf.float32)

In [5]:
def augment_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    def apply_augmentation(img):
        choice = tf.random.uniform(shape=[], minval=0, maxval=7, dtype=tf.int32)
        
        # All augmentation functions now return float32
        def apply_flip(): return random_flip(img)
        def apply_brightness(): return tf.image.random_brightness(img, 0.4)
        def apply_contrast(): return random_contrast(img)
        def apply_rotation(): return random_rotation(tf.expand_dims(img, 0))[0]
        def apply_zoom(): return random_zoom(tf.expand_dims(img, 0))[0]
        def apply_hue(): return tf.image.random_hue(img, 0.3)
        def apply_saturation(): return tf.image.random_saturation(img, 0.6, 1.6)
            
        return tf.switch_case(choice, [
            apply_flip, apply_brightness, apply_contrast, apply_rotation,
            apply_zoom, apply_hue, apply_saturation
        ])

    # 50% chance to apply augmentation
    image = tf.cond(
        tf.random.uniform(()) < 0.5,
        lambda: apply_augmentation(image),
        lambda: image
    )
    
    image = preprocess_input(image)
    return tf.cast(image, tf.float16), label

In [6]:
# Load dataset using tf.data
AUTOTUNE = tf.data.AUTOTUNE

In [7]:
def create_dataset(directory, augment=False):
    ds = tf.keras.utils.image_dataset_from_directory(
        directory,
        image_size=(256, 256),
        batch_size=None,
        shuffle=augment,
        seed=42 if augment else None
    )
    if augment:
        ds = ds.map(augment_image, num_parallel_calls=AUTOTUNE)
    else:
        ds = ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
    return ds.batch(64).prefetch(AUTOTUNE)

In [8]:
# Download dataset
import kagglehub
dataset_path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")
base_dir = os.path.join(dataset_path, "Dataset")

train_ds = create_dataset(os.path.join(base_dir, "Train"), augment=True)
val_ds = create_dataset(os.path.join(base_dir, "Validation"))
test_ds = create_dataset(os.path.join(base_dir, "Test"))

Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [9]:
# Model setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid', dtype='float32')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_auc', mode='max', factor=0.5, patience=2, min_lr=1e-7, verbose=1
)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "best_model.keras", monitor='val_auc', mode='max', save_best_only=True, verbose=1
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', mode='max', patience=5, restore_best_weights=True, verbose=1
)

In [11]:
# Phase 1: Frozen base
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

In [12]:
history_phase1 = model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr]
)

Epoch 1/30
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.7533 - auc: 0.8353 - loss: 4.1433
Epoch 1: val_auc improved from -inf to 0.90865, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 579s 257ms/step - accuracy: 0.7533 - auc: 0.8353 - loss: 4.1424 - val_accuracy: 0.8276 - val_auc: 0.9086 - val_loss: 0.7838 - learning_rate: 1.0000e-04
Epoch 2/30
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.8185 - auc: 0.9026 - loss: 0.6950
Epoch 2: val_auc improved from 0.90865 to 0.91364, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 484s 221ms/step - accuracy: 0.8185 - auc: 0.9026 - loss: 0.6950 - val_accuracy: 0.8275 - val_auc: 0.9136 - val_loss: 0.5311 - learning_rate: 1.0000e-04
Epoch 3/30
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.8245 - auc: 0.9090 - loss: 0.5125
Epoch 3: val_auc improved from 0.91364 to 0.91663, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 482s 220ms/step - accuracy: 0.8245 - auc: 

In [13]:
# Evaluation
model.load_weights("best_model.keras")
test_loss, test_acc, test_auc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

171/171 ━━━━━━━━━━━━━━━━━━━━ 28s 161ms/step - accuracy: 0.8078 - auc: 0.4273 - loss: 0.4677
Test Loss: 0.6804
Test Accuracy: 0.7353
Test AUC: 0.8240


In [14]:
# Phase 2: Fine-tuning
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

In [15]:
history_phase2 = model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr]
)

Epoch 1/15
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step - accuracy: 0.9356 - auc: 0.9825 - loss: 0.1857
Epoch 1: val_auc improved from 0.93865 to 0.99487, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1053s 444ms/step - accuracy: 0.9356 - auc: 0.9825 - loss: 0.1857 - val_accuracy: 0.9700 - val_auc: 0.9949 - val_loss: 0.1102 - learning_rate: 1.0000e-05
Epoch 2/15
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.9810 - auc: 0.9981 - loss: 0.0757
Epoch 2: val_auc improved from 0.99487 to 0.99582, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 920s 420ms/step - accuracy: 0.9810 - auc: 0.9981 - loss: 0.0757 - val_accuracy: 0.9713 - val_auc: 0.9958 - val_loss: 0.1021 - learning_rate: 1.0000e-05
Epoch 3/15
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - accuracy: 0.9876 - auc: 0.9993 - loss: 0.0526
Epoch 3: val_auc improved from 0.99582 to 0.99701, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 925s 423ms/step - accuracy: 0.9876 - a

In [16]:
# Evaluation
model.load_weights("best_model.keras")
test_loss, test_acc, test_auc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

171/171 ━━━━━━━━━━━━━━━━━━━━ 18s 107ms/step - accuracy: 0.9338 - auc: 0.4717 - loss: 0.2856
Test Loss: 0.8210
Test Accuracy: 0.8386
Test AUC: 0.9047
